<a href="https://colab.research.google.com/github/prateekchandrajha/worldquant-online-algorithms/blob/main/Follow_The_Leader_Algorithms_FTRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import math
import numpy as np

random.seed(1)

References for FTRL Algorithm:

- "Follow the Regularized Leader" method provides a framework to design and analyze online algorithms in a versatile and well-motivated way. We will be using FTRL to simulate trading before the final submission in February.

- https://medium.com/@dhirajreddy13/factorization-machines-and-follow-the-regression-leader-for-dummies-7657652dce69

- FTRL and Mirror Descent Algorithms: https://storage.googleapis.com/pub-tools-public-publication-data/pdf/37013.pdf

In [2]:
class FollowTheRegularizedLeader(object):
    def __init__(self, alpha, beta, l1, l2, features, reg_func, indices):
        self.alpha = alpha
        self.beta = beta
        self.l1 = l1
        self.l2 = l2
        self.reg_func = reg_func
        self.indices = [0]
        self.count = 0
        for index in indices:
            self.indices.append(index)
        self.sum_of_gradients_squared = [0.] * features
        self.weights = {}
        self.temp_weights = []
        i = 0
        while i < features:
            self.temp_weights.append(random.random())
            i += 1

    def update(self, probability, result):
        gradient = probability - result
        gradient_squared = math.pow(gradient, 2)
        self.count += 1
        if self.reg_func is "RDA":
            for i in self.indices:
                sigma = (math.sqrt(self.sum_of_gradients_squared[i] + gradient_squared)) / (self.alpha * self.count)
                self.temp_weights[i] += -(sigma * self.weights[i]) + gradient
                self.sum_of_gradients_squared[i] += gradient_squared
        elif self.reg_func is "OPG":
            for i in self.indices:
                sigma = (math.sqrt(self.sum_of_gradients_squared[i] + gradient_squared) - math.sqrt(self.sum_of_gradients_squared[i])) / self.alpha
                self.temp_weights[i] += -(sigma * self.weights[i]) + gradient
                self.sum_of_gradients_squared[i] += gradient_squared

    def predict(self):
        weights = {}
        function = "Sigmoid"
        w_inner_x = float(0)
        for i in self.indices:
            sign = float(np.sign(self.temp_weights))
            if sign * self.temp_weights[i] <= self.l1:
                weights[i] = float(0)
            else:
                weights[i] = (sign * self.l1 - self.temp_weights[i]) / ((self.beta + math.sqrt(self.sum_of_gradients_squared[i])) / self.alpha + self.l2)
            w_inner_x += weights[i]

        self.weights = weights

        if function is "Sigmoid":
            probability = float(1) / (float(1) + math.exp(-max(min(float(w_inner_x), float(100)), float(-100))))
        elif function is "ReLU":
            probability = max(float(0), max(min(float(w_inner_x), float(100)), float(-100)))

        return probability

    @staticmethod
    def log_loss(true_label, predicted, eps=1e-15):
        p = np.clip(predicted, eps, 1 - eps)
        if true_label > 0:
            return -math.log(p)
        else:
            return -math.log(1 - p)

In [3]:
# Declaring FTRL Parameters

alpha = 0.2
beta = 0.5
l1 = 0.32
l2 = 0.32
# features = number of features
# reg_func = either "RDA" or "OPG"
# indices

FTRL_simulate = FollowTheRegularizedLeader(alpha, beta, l1, l2, features, reg_func, indices) # declaring an object of FTRL algorithm class for performing simulations

NameError: ignored

## Adding Epsilon-Greedy Bandit Algorithm to the Mix

In [25]:
import random

random.seed(1)


class EpsilonGreedy:
    def __init__(self, epsilon, pulls_per_arm, avg_vals):
        self.epsilon = epsilon  # probability of explore
        self.pulls_per_arm = pulls_per_arm  # number of pulls for each arms
        self.avg_vals = avg_vals  # average amount of reward we've gotten from each arms
        return

    def initialize(self, n_arms):
        self.pulls_per_arm = [0] * n_arms
        self.avg_vals = [0.0] * n_arms
        return

    def arm_selection(self):
        # For 1-epsilon probability the current average maximum valued arm will be chosen
        if random.random() > self.epsilon:
            return self.avg_vals.index(max(self.avg_vals))
        # A random other arm will be chosen
        else:
            return random.randrange(len(self.avg_vals))

    def update(self, chosen_arm, reward):
        avg_val = self.avg_vals[chosen_arm]
        self.pulls_per_arm[chosen_arm] += 1
        pulls = self.pulls_per_arm[chosen_arm]
        updated_val = (((pulls - 1) * avg_val) + reward) / float(pulls)
        self.avg_vals[chosen_arm] = updated_val
        return


class StockHistoryArm:
    def __init__(self, gains):
        self.gains = gains

    def draw(self, ind):
        return self.gains[ind]

## Importing Data From AlphaVantage

In [21]:
import json
import urllib.request

def JSON_REQUEST(ticker, output_size):
    # handle = json.load(api_key_file)
    apikey = "P4W07RP50IIDXWVR"
    url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=" + ticker + "&outputsize=" + output_size + "&apikey=" + apikey
    data = urllib.request.urlopen(url)
    output = json.load(data)
    # print(output)
    filename = "/content/data/JSON/" + ticker + "_stock_data.json"
    with open(filename, 'w') as outfile:
        json.dump(output, outfile, indent=4)

In [18]:
!pwd

/content


## JSON Functions

In [22]:
import json
from pathlib import Path

class StockInfo:
    def __init__(self, ticker, entries):
        self.ticker = ticker
        self.entries = entries
        self.filename = "/content/data/JSON/" + self.ticker + "_stock_data.json"
        self.total_cumulative_gain_string = ""
        self.stock_time_interval = []
        self.gain_list = []
        self.cumulative_gain_list = []
        self.calculate_gains()
        self.calculate_cumulative_gain()

    def find_dates(self):
        try:
            my_abs_path = Path(self.filename).resolve(strict=True)
        except FileNotFoundError:
            print("File does not exist - calling API")
            JSON_REQUEST(self.ticker, "full")
        else:
            with open(self.filename) as json_data:
                handle = json.load(json_data)
                i = 0
                for data in handle["Time Series (Daily)"]:
                    if i < self.entries:
                        self.stock_time_interval.append(data)
                    i += 1
                self.stock_time_interval.reverse()

    def calculate_gains(self):
        i = 0
        previous = 0
        self.find_dates()
        with open(self.filename) as json_data:
            handle = json.load(json_data)
            for day in self.stock_time_interval:
                current = handle["Time Series (Daily)"][day]["4. close"]
                if i == 0:
                    previous = handle["Time Series (Daily)"][day]["4. close"]
                else:
                    gains = ((float(current) - float(previous)) / float(previous))
                    previous = handle["Time Series (Daily)"][day]["4. close"]
                    self.gain_list.append(gains)
                i += 1

    def calculate_cumulative_gain(self):
        i = 0
        cumulative_gain = 0
        for gain in self.gain_list:
            current = gain
            if i == 0:
                previous = gain
                cumulative_gain += gain
            else:
                cumulative_gain += current + (current * previous)
                previous = cumulative_gain
            i += 1
            self.cumulative_gain_list.append(cumulative_gain)
        self.total_cumulative_gain_string = str(round(self.cumulative_gain_list[-1] * 100, 4)) + '%'

## For Proper Formatting of Reports

In [6]:
def epsilon_greedy_output_formatted(epsilon_greedy_instance, arms, number_of_entries):
    iterator = [0.0] * number_of_entries
    chosen_arms = [0.0] * number_of_entries
    rewards = [0.0] * number_of_entries
    cumulative_rewards = [0.0] * number_of_entries
    epsilon_greedy_instance.initialize(len(arms))

    i = 0
    while i < number_of_entries:
        iterator[i] = i + 1
        chosen_arm = epsilon_greedy_instance.arm_selection()
        chosen_arms[i] = chosen_arm
        reward = arms[chosen_arm].draw(i)
        rewards[i] = reward

        if i == 0:
            cumulative_rewards[i] = reward
        else:
            cumulative_rewards[i] = cumulative_rewards[i - 1] + reward + (cumulative_rewards[i - 1] * reward)

        epsilon_greedy_instance.update(chosen_arm, reward)
        i += 1

    return [iterator, chosen_arms, rewards, cumulative_rewards]


def ftrl_output_formatted(ftrl_model_set, stock_set, number_of_entries):
    iterator = [0.0] * number_of_entries
    chosen_stocks = [0.0] * number_of_entries
    rewards = [0.0] * number_of_entries
    cumulative_rewards = [0.0] * number_of_entries

    possible_rewards = [0.0] * len(stock_set)
    losses = [0.0] * len(stock_set)
    probabilities = [0.0] * len(stock_set)

    i = 0
    while i < number_of_entries:
        iterator[i] = i + 1
        j = 0
        while j < len(stock_set):
            possible_rewards[j] = stock_set[j][i]
            probabilities[j] = ftrl_model_set[j].predict()
            losses[j] += FollowTheRegularizedLeader.log_loss(stock_set[j][i], probabilities[j])
            ftrl_model_set[j].update(probabilities[j], stock_set[j][i])
            j += 1
        index_of_maximum_probability = probabilities.index(max(probabilities))
        chosen_stocks[i] = index_of_maximum_probability
        rewards[i] = possible_rewards[index_of_maximum_probability]

        if i == 0:
            cumulative_rewards[i] = rewards[i]
        else:
            cumulative_rewards[i] = cumulative_rewards[i - 1] + rewards[i] + (cumulative_rewards[i - 1] * rewards[i])
        i += 1

    return [iterator, chosen_stocks, rewards, cumulative_rewards]

## Running Our Algorithms on Stock Data Downloaded From AlphaVantage

In [29]:
number_of_entries = 1000

apple = StockInfo("AAPL", number_of_entries)
print(apple.ticker + ": " + apple.total_cumulative_gain_string)

microsoft = StockInfo("MSFT", number_of_entries)
print(microsoft.ticker + ": " + microsoft.total_cumulative_gain_string)

intel = StockInfo("INTC", number_of_entries)
print(intel.ticker + ": " + intel.total_cumulative_gain_string)

google = StockInfo("GOOGL", number_of_entries)
print(google.ticker + ": " + google.total_cumulative_gain_string)

amazon = StockInfo("AMZN", number_of_entries)
print(amazon.ticker + ": " + amazon.total_cumulative_gain_string)

stock_set_increasing = [apple.gain_list, google.gain_list, amazon.gain_list, intel.gain_list, microsoft.gain_list]
arms = list(map(lambda mu: StockHistoryArm(mu), stock_set_increasing))

f = open("/content/data/output/epsilon_greedy_increasing_stocks.csv", "w")

for epsilon in [0.1, 0.2, 0.3, 0.4, 0.5]:
    epsilon_greedy_instance = EpsilonGreedy(epsilon, [], [])
    results = epsilon_greedy_output_formatted(epsilon_greedy_instance, arms, number_of_entries - 1)
    for i in range(len(results[0])):
        f.write(str(epsilon) + ',' + str(results[0][i]) + ',' + str(results[1][i]) + ',' + str(round(results[2][i], 4)) + ',' + str(round(results[3][i], 4)) + '\n')

f.close()

f = open("/content/data/output/ftrl_increasing_stocks_opg.csv", "w")

ftr1_instance1 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])
ftr1_instance2 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])
ftr1_instance3 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])
ftr1_instance4 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])
ftr1_instance5 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])

ftrl_instances = [ftr1_instance1, ftr1_instance2, ftr1_instance3, ftr1_instance4, ftr1_instance5]

results = ftrl_output_formatted(ftrl_instances, stock_set_increasing, number_of_entries - 1)

for i in range(len(results[0])):
    f.write(str(results[0][i]) + "," + str(results[1][i]) + "," + str(round(results[2][i], 4)) + "," + str(round(results[3][i], 4)) + "\n")

del ftr1_instance1
del ftr1_instance2
del ftr1_instance3
del ftr1_instance4
del ftr1_instance5
del ftrl_instances
del results

f.close()

f = open("/content/data/output/ftl_increasing_stocks.csv", "w")

ftr1_instance1 = FollowTheRegularizedLeader(0.05, 1.0, 0.0, 0.0, 1, "OPG", [0])
ftr1_instance2 = FollowTheRegularizedLeader(0.05, 1.0, 0.0, 0.0, 1, "OPG", [0])
ftr1_instance3 = FollowTheRegularizedLeader(0.05, 1.0, 0.0, 0.0, 1, "OPG", [0])
ftr1_instance4 = FollowTheRegularizedLeader(0.05, 1.0, 0.0, 0.0, 1, "OPG", [0])
ftr1_instance5 = FollowTheRegularizedLeader(0.05, 1.0, 0.0, 0.0, 1, "OPG", [0])

ftrl_instances = [ftr1_instance1, ftr1_instance2, ftr1_instance3, ftr1_instance4, ftr1_instance5]

results = ftrl_output_formatted(ftrl_instances, stock_set_increasing, number_of_entries - 1)

for i in range(len(results[0])):
    f.write(str(results[0][i]) + "," + str(results[1][i]) + "," + str(round(results[2][i], 4)) + "," + str(round(results[3][i], 4)) + "\n")

del ftr1_instance1
del ftr1_instance2
del ftr1_instance3
del ftr1_instance4
del ftr1_instance5
del ftrl_instances
del results

f.close()

f = open("/content/data/output/ftrl_increasing_stocks_rda.csv", "w")

ftr1_instance1 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])
ftr1_instance2 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])
ftr1_instance3 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])
ftr1_instance4 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])
ftr1_instance5 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])

ftrl_instances = [ftr1_instance1, ftr1_instance2, ftr1_instance3, ftr1_instance4, ftr1_instance5]

results = ftrl_output_formatted(ftrl_instances, stock_set_increasing, number_of_entries - 1)

for i in range(len(results[0])):
    f.write(str(results[0][i]) + "," + str(results[1][i]) + "," + str(round(results[2][i], 4)) + "," + str(round(results[3][i], 4)) + "\n")

del ftr1_instance1
del ftr1_instance2
del ftr1_instance3
del ftr1_instance4
del ftr1_instance5
del ftrl_instances

f.close()


number_of_entries = 950

twitter = StockInfo("TWTR", number_of_entries)
print(twitter.ticker + ": " + twitter.total_cumulative_gain_string)

tesla = StockInfo("TSLA", number_of_entries)
print(tesla.ticker + ": " + tesla.total_cumulative_gain_string)

fitbit = StockInfo("FIT", number_of_entries)
print(fitbit.ticker + ": " + fitbit.total_cumulative_gain_string)

altaba = StockInfo("AABA", number_of_entries)
print(altaba.ticker + ": " + altaba.total_cumulative_gain_string)

general_electric = StockInfo("GE", number_of_entries)
print(general_electric.ticker + ": " + general_electric.total_cumulative_gain_string)

stock_set_mixed = [twitter.gain_list, altaba.gain_list, general_electric.gain_list, fitbit.gain_list, tesla.gain_list]
arms = list(map(lambda mu: StockHistoryArm(mu), stock_set_mixed))


f = open("/content/data/output/epsilon_greedy_mixed_stocks.csv", "w")

for epsilon in [0.1, 0.2, 0.3, 0.4, 0.5]:
    epsilon_greedy_instance = EpsilonGreedy(epsilon, [], [])
    results = epsilon_greedy_output_formatted(epsilon_greedy_instance, arms, number_of_entries - 1)
    for i in range(len(results[0])):
        f.write(str(epsilon) + ',' + str(results[0][i]) + ',' + str(results[1][i]) + ',' + str(round(results[2][i], 4)) + ',' + str(round(results[3][i], 4)) + '\n')

f.close()

f = open("/content/data/output/ftrl_mixed_stocks_opg.csv", "w")

ftrl_instance6 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])
ftrl_instance7 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])
ftrl_instance8 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])
ftrl_instance9 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])
ftrl_instance10 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])

ftrl_instances2 = [ftrl_instance6, ftrl_instance7, ftrl_instance8, ftrl_instance9, ftrl_instance10]

results = ftrl_output_formatted(ftrl_instances2, stock_set_mixed, number_of_entries - 1)

for i in range(len(results[0])):
    f.write(str(results[0][i]) + "," + str(results[1][i]) + "," + str(round(results[2][i], 4)) + "," + str(round(results[3][i], 4)) + "\n")

del ftrl_instance6
del ftrl_instance7
del ftrl_instance8
del ftrl_instance9
del ftrl_instance10
del ftrl_instances2
del results

f.close()


f = open("/content/data/output/ftl_mixed_stocks.csv", "w")

ftrl_instance6 = FollowTheRegularizedLeader(0.05, 1.0, 0.0, 0.0, 1, "OPG", [0])
ftrl_instance7 = FollowTheRegularizedLeader(0.05, 1.0, 0.0, 0.0, 1, "OPG", [0])
ftrl_instance8 = FollowTheRegularizedLeader(0.05, 1.0, 0.0, 0.0, 1, "OPG", [0])
ftrl_instance9 = FollowTheRegularizedLeader(0.05, 1.0, 0.0, 0.0, 1, "OPG", [0])
ftrl_instance10 = FollowTheRegularizedLeader(0.05, 1.0, 0.0, 0.0, 1, "OPG", [0])

ftrl_instances2 = [ftrl_instance6, ftrl_instance7, ftrl_instance8, ftrl_instance9, ftrl_instance10]

results = ftrl_output_formatted(ftrl_instances2, stock_set_mixed, number_of_entries - 1)

for i in range(len(results[0])):
    f.write(str(results[0][i]) + "," + str(results[1][i]) + "," + str(round(results[2][i], 4)) + "," + str(round(results[3][i], 4)) + "\n")

del ftrl_instance6
del ftrl_instance7
del ftrl_instance8
del ftrl_instance9
del ftrl_instance10
del ftrl_instances2
del results

f.close()

f = open("/content/data/output/ftrl_mixed_stocks_rda.csv", "w")

ftrl_instance6 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "RDA", [0])
ftrl_instance7 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "RDA", [0])
ftrl_instance8 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "RDA", [0])
ftrl_instance9 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "RDA", [0])
ftrl_instance10 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "RDA", [0])

ftrl_instances2 = [ftrl_instance6, ftrl_instance7, ftrl_instance8, ftrl_instance9, ftrl_instance10]

results = ftrl_output_formatted(ftrl_instances2, stock_set_mixed, number_of_entries - 1)

for i in range(len(results[0])):
    f.write(str(results[0][i]) + "," + str(results[1][i]) + "," + str(round(results[2][i], 4)) + "," + str(round(results[3][i], 4)) + "\n")

del ftrl_instance6
del ftrl_instance7
del ftrl_instance8
del ftrl_instance9
del ftrl_instance10
del ftrl_instances2
del results

f.close()

number_of_entries = 950

qualcomm = StockInfo("QCOM", number_of_entries)
print(qualcomm.ticker + ": " + qualcomm.total_cumulative_gain_string)

sony = StockInfo("SNE", number_of_entries)
print(sony.ticker + ": " + sony.total_cumulative_gain_string)

cisco = StockInfo("CSCO", number_of_entries)
print(cisco.ticker + ": " + cisco.total_cumulative_gain_string)

activision = StockInfo("ATVI", number_of_entries)
print(activision.ticker + ": " + activision.total_cumulative_gain_string)

xilinx = StockInfo("XLNX", number_of_entries)
print(xilinx.ticker + ": " + xilinx.total_cumulative_gain_string)

stock_set_big = [apple.gain_list, google.gain_list, amazon.gain_list, intel.gain_list, microsoft.gain_list, twitter.gain_list, altaba.gain_list, general_electric.gain_list, fitbit.gain_list, tesla.gain_list, qualcomm.gain_list, sony.gain_list, cisco.gain_list, activision.gain_list, xilinx.gain_list]
arms = list(map(lambda mu: StockHistoryArm(mu), stock_set_big))

f = open("/content/data/output/epsilon_greedy_big_stocks.csv", "w")

for epsilon in [0.1, 0.2, 0.3, 0.4, 0.5]:
    epsilon_greedy_instance = EpsilonGreedy(epsilon, [], [])
    results = epsilon_greedy_output_formatted(epsilon_greedy_instance, arms, number_of_entries - 1)
    for i in range(len(results[0])):
        f.write(str(epsilon) + ',' + str(results[0][i]) + ',' + str(results[1][i]) + ',' + str(round(results[2][i], 4)) + ',' + str(round(results[3][i], 4)) + '\n')

f.close()

f = open("/content/data/output/ftl_big_stocks.csv", "w")

ftrl_instance1 = FollowTheRegularizedLeader(0.05, 1.0, 0.0, 0.0, 1, "OPG", [0])
ftrl_instance2 = FollowTheRegularizedLeader(0.05, 1.0, 0.0, 0.0, 1, "OPG", [0])
ftrl_instance3 = FollowTheRegularizedLeader(0.05, 1.0, 0.0, 0.0, 1, "OPG", [0])
ftrl_instance4 = FollowTheRegularizedLeader(0.05, 1.0, 0.0, 0.0, 1, "OPG", [0])
ftrl_instance5 = FollowTheRegularizedLeader(0.05, 1.0, 0.0, 0.0, 1, "OPG", [0])
ftrl_instance6 = FollowTheRegularizedLeader(0.05, 1.0, 0.0, 0.0, 1, "OPG", [0])
ftrl_instance7 = FollowTheRegularizedLeader(0.05, 1.0, 0.0, 0.0, 1, "OPG", [0])
ftrl_instance8 = FollowTheRegularizedLeader(0.05, 1.0, 0.0, 0.0, 1, "OPG", [0])
ftrl_instance9 = FollowTheRegularizedLeader(0.05, 1.0, 0.0, 0.0, 1, "OPG", [0])
ftrl_instance10 = FollowTheRegularizedLeader(0.05, 1.0, 0.0, 0.0, 1, "OPG", [0])
ftrl_instance11 = FollowTheRegularizedLeader(0.05, 1.0, 0.0, 0.0, 1, "OPG", [0])
ftrl_instance12 = FollowTheRegularizedLeader(0.05, 1.0, 0.0, 0.0, 1, "OPG", [0])
ftrl_instance13 = FollowTheRegularizedLeader(0.05, 1.0, 0.0, 0.0, 1, "OPG", [0])
ftrl_instance14 = FollowTheRegularizedLeader(0.05, 1.0, 0.0, 0.0, 1, "OPG", [0])
ftrl_instance15 = FollowTheRegularizedLeader(0.05, 1.0, 0.0, 0.0, 1, "OPG", [0])

ftrl_instances3 = [ftrl_instance1, ftrl_instance2, ftrl_instance3, ftrl_instance4, ftrl_instance5, ftrl_instance6, ftrl_instance7, ftrl_instance8, ftrl_instance9, ftrl_instance10, ftrl_instance11, ftrl_instance12, ftrl_instance13, ftrl_instance14, ftrl_instance15]

results = ftrl_output_formatted(ftrl_instances3, stock_set_big, number_of_entries - 1)

for i in range(len(results[0])):
    f.write(str(results[0][i]) + "," + str(results[1][i]) + "," + str(round(results[2][i], 4)) + "," + str(round(results[3][i], 4)) + "\n")

del ftrl_instance1
del ftrl_instance2
del ftrl_instance3
del ftrl_instance4
del ftrl_instance5
del ftrl_instance6
del ftrl_instance7
del ftrl_instance8
del ftrl_instance9
del ftrl_instance10
del ftrl_instance11
del ftrl_instance12
del ftrl_instance13
del ftrl_instance14
del ftrl_instance15
del ftrl_instances3
del results

f.close()


f = open("/content/data/output/ftrl_big_stocks_rda.csv", "w")

ftrl_instance1 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "RDA", [0])
ftrl_instance2 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "RDA", [0])
ftrl_instance3 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "RDA", [0])
ftrl_instance4 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "RDA", [0])
ftrl_instance5 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "RDA", [0])
ftrl_instance6 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "RDA", [0])
ftrl_instance7 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "RDA", [0])
ftrl_instance8 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "RDA", [0])
ftrl_instance9 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "RDA", [0])
ftrl_instance10 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "RDA", [0])
ftrl_instance11 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "RDA", [0])
ftrl_instance12 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "RDA", [0])
ftrl_instance13 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "RDA", [0])
ftrl_instance14 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "RDA", [0])
ftrl_instance15 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "RDA", [0])

ftrl_instances3 = [ftrl_instance1, ftrl_instance2, ftrl_instance3, ftrl_instance4, ftrl_instance5, ftrl_instance6, ftrl_instance7, ftrl_instance8, ftrl_instance9, ftrl_instance10, ftrl_instance11, ftrl_instance12, ftrl_instance13, ftrl_instance14, ftrl_instance15]

results = ftrl_output_formatted(ftrl_instances3, stock_set_big, number_of_entries - 1)

for i in range(len(results[0])):
    f.write(str(results[0][i]) + "," + str(results[1][i]) + "," + str(round(results[2][i], 4)) + "," + str(round(results[3][i], 4)) + "\n")

del ftrl_instance1
del ftrl_instance2
del ftrl_instance3
del ftrl_instance4
del ftrl_instance5
del ftrl_instance6
del ftrl_instance7
del ftrl_instance8
del ftrl_instance9
del ftrl_instance10
del ftrl_instance11
del ftrl_instance12
del ftrl_instance13
del ftrl_instance14
del ftrl_instance15
del ftrl_instances3
del results

f.close()

f = open("/content/data/output/ftrl_big_stocks_opg.csv", "w")

ftrl_instance1 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])
ftrl_instance2 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])
ftrl_instance3 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])
ftrl_instance4 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])
ftrl_instance5 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])
ftrl_instance6 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])
ftrl_instance7 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])
ftrl_instance8 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])
ftrl_instance9 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])
ftrl_instance10 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])
ftrl_instance11 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])
ftrl_instance12 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])
ftrl_instance13 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])
ftrl_instance14 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])
ftrl_instance15 = FollowTheRegularizedLeader(0.05, 1.0, 1.0, 1.0, 1, "OPG", [0])

ftrl_instances3 = [ftrl_instance1, ftrl_instance2, ftrl_instance3, ftrl_instance4, ftrl_instance5, ftrl_instance6, ftrl_instance7, ftrl_instance8, ftrl_instance9, ftrl_instance10, ftrl_instance11, ftrl_instance12, ftrl_instance13, ftrl_instance14, ftrl_instance15]

results = ftrl_output_formatted(ftrl_instances3, stock_set_big, number_of_entries - 1)

for i in range(len(results[0])):
    f.write(str(results[0][i]) + "," + str(results[1][i]) + "," + str(round(results[2][i], 4)) + "," + str(round(results[3][i], 4)) + "\n")

del ftrl_instance1
del ftrl_instance2
del ftrl_instance3
del ftrl_instance4
del ftrl_instance5
del ftrl_instance6
del ftrl_instance7
del ftrl_instance8
del ftrl_instance9
del ftrl_instance10
del ftrl_instance11
del ftrl_instance12
del ftrl_instance13
del ftrl_instance14
del ftrl_instance15
del ftrl_instances3
del results

f.close()

AAPL: 1.5289%
MSFT: 153.639%
INTC: 79.6928%
GOOGL: 126.2277%
AMZN: 341.3889%
TWTR: 7.9866%
TSLA: -16.6938%
FIT: -87.9469%
AABA: 92.5139%
GE: -64.7384%
QCOM: 34.1798%
SNE: 77.4841%
CSCO: 103.2328%
ATVI: 85.7803%
XLNX: 192.525%


## Plotting

In [34]:
import pandas as pd
from datetime import datetime
import csv
import matplotlib.pyplot as plt
# import matplotlib.dates as mdates

# headers = ['Sensor Value','Date','Time']
df = pd.read_csv('/content/data/output/ftl_increasing_stocks.csv')#,names=headers)
print (df)

       1  4  0.0003  0.0003.1
0      2  4 -0.0085   -0.0082
1      3  4 -0.0133   -0.0214
2      4  4 -0.0277   -0.0485
3      5  4  0.0091   -0.0399
4      6  4  0.0225   -0.0183
..   ... ..     ...       ...
993  995  4  0.0052   -0.2234
994  996  4  0.0008   -0.2227
995  997  4 -0.0023   -0.2245
996  998  4  0.0083   -0.2181
997  999  4  0.0131   -0.2078

[998 rows x 4 columns]
